# Tutorial
JupyterJS heavily rely on communications between python and javascript kernels, which are asynchronous. It is strongly recommended to execute cells one-by-one instead of running all cells.  
It will be helpful to familiar yourself with common asynchronous behaviors in Jupyter, e.g.,:  
- [print() does not work](https://github.com/jupyter-widgets/ipywidgets/issues/2121)

In [1]:
from jupyterjs import JupyterJS
import ipywidgets as widgets

## 0. Base Usage

In [2]:
w = JupyterJS()
w.display()

Expect to see an empty display with a console.
![Basic Usage](basicusage.png)

## 1. Map Variable and Functions
JupyterJS provides an interface to bind variables/functions between the JS and Py scope. In other words, both JS/Py kernels have access to those variables/functions.

### 1.1 Variables
JupyterJS provides a *mapstate* method to map state variables between JS and Python.
This method must be declared within the constructor.

- Variables declared in python should be in a dict syntax with its initial value.
- Variables declared in JavaScript are as string.

In [3]:
class VariableMapper(JupyterJS):
    mapstate = [{'pyv': 5}, 'jsv']
w = VariableMapper()
w.display()

Expect to see the following result. Note that now the variable *pyv* has a value as 5. The variable *jsv* is None because it has not been found in JS programs.
![Expect to see the following result](1.1.png)



In [4]:
# Now the JS scope can access pyv. Run this cell and you will see an altet window!
w.javascript("alert(pyv)")

Variables declared in mapstate() are binded to JupyterJS.state. 

In [5]:
# Now change its value in JS.
w.javascript("pyv = pyv + 5")

Expect to see the following result in the **previous console**. Note that a JupyterJS object is binded to **only one display area** with JupyterJS.display(). Now you can see *pyv* has a value as 10.
![Example 1.2](1.2.png)



In [6]:
## Verify the results
print(w.state.pyv)

10


Recall that jsv has not been found! Now let us define it in JavaScript.

In [8]:
code = """
    var jsv = 'abc';
"""
w.javascript(code)
print(w.state.jsv)

abc


Variables in mapstate() need to be **JSON-safe**, e.g., numbers, strings, dicts, and lists.  
**Classes are not supported**!

### 1.2 Functions
The *mapmethod* method provides a convenient way to bind methods. Similarly, it takes an list of single-key object or string.

In [9]:
out = widgets.Output()
display(out)

Output()

In [10]:
def foo(a):
    with out:
        print('this is a py function:', a * 2)

class MethodMapper(JupyterJS):
    mapstate = [{'pyv': 1}]
    mapmethod = [{'pyfunc': foo}, 'jsfunc']
    
methodw = MethodMapper()
methodw.display()

Similarly, now you can see pyfunc and jsfunc in the console.
![Example 1.3](1.3.png)

Now you can run pyfunc from JS.

In [11]:
code = """
    pyfunc(pyv)
"""
methodw.javascript(code)

Check the python console area!
![Example 1.4](1.4.png)

Similarly, declare *jsfunc* and run it from python! Note that methods in *mapmethod* is attached to JupyterJS.methods.

In [12]:
code = """
    const jsfunc = (x, y) => {
        alert(`this is a js function called from py: ${x * 3}, ${y * 4}`);
    }
"""
methodw.javascript(code)

In [13]:
methodw.methods.jsfunc(1, 5)

The above is the same as

In [14]:
methodw.javascript('jsfunc(1,5)')

### 1.3 Scope
Variables outside mapstate() will lost their scope. 
This is the default behavior of the built-in IPython.display.Javascript().

In [15]:
from IPython.display import Javascript
Javascript("var t = 5;")

<IPython.core.display.Javascript object>

In [16]:
## you will see errors 
Javascript("t = t + 10")

<IPython.core.display.Javascript object>

This is the same for JupyterJS.

In [17]:
class ScopeTester(JupyterJS):
    mapstate = ['jsv1']
    
stest = ScopeTester()
stest.display()

In [18]:
stest.javascript("var jsv1 = 2; var jsv2 = 4;") 

In [19]:
## This will work
stest.javascript("alert(jsv1)")

In [20]:
## This will work not
stest.javascript("alert(jsv2)")

### 1.4 Summary
Now putting everything together. Run the following codes and play yourself!

In [21]:
def foo(a, b):
    print('this is a py function call from JS', a * 2, b)
    
class JSWidget(JupyterJS):
    mapstate = [{'pyv': 5}, 'jsv']
    mapmethod = [{'pyf': foo}, 'jsf']

w = JSWidget()
w.display()

js = """
    pyf(4, 3);
    
    var jsv = 5;
    
    jsv = jsv * 2;
    pyv = 'abc'
    
    const jsf = (x, y) => {
        console.log('this is a js function called from py: ', x * 3, y);
    }
"""

In [22]:
w.javascript(js)

In [23]:
w.javascript("pyv = 2; alert(pyv * jsv);")

## 2 Import JS packages
JupyterJS provides helper functions to import es6 and umd packages.  
Check out this [article](https://blog.sessionstack.com/how-javascript-works-the-module-pattern-comparing-commonjs-amd-umd-and-es6-modules-437f77548437) for different javascript modules formats.  
⚠️ Importing UMD packages are only allowed in Jupyter Lab (no Notebooks). See https://github.com/jupyterlab/jupyterlab/issues/3118.


### 2.1 Import ES6
The *import_es6* helper takes two input: the import command, and the import name.  

In [24]:
class D3Widget(JupyterJS):
    def __init__(self, *pargs, **kwargs):
        super(D3Widget, self).__init__(*pargs, **kwargs)        
        self.import_es6('import * as d3 from "https://cdn.jsdelivr.net/npm/d3@7/+esm";', 'd3')
        
d3w = D3Widget()
d3w.display()

In [25]:
w.javascript("alert(d3.color.name)")

### 2.2 Import UMD
The *import_umd* helper takes two input: the script url, and the import name.  
⚠️ This does not work in Jupyter Notebooks, but Jupyter Lab only!

In [26]:
class DeckGLWidget(JupyterJS):
    def __init__(self, *pargs, **kwargs):
        super(DeckGLWidget, self).__init__(*pargs, **kwargs)        
        self.import_umd("https://unpkg.com/deck.gl@8.8.23/dist.min.js", 'deck')
        
deckw = DeckGLWidget()
deckw.display()

In [27]:
deckw.javascript("alert(deck.version)")

## 3. Build a customize JS visualization
Now you have known most of JupyterJS. The following text provides a hand-on tutorial for building a custom vis!   
JupyterJS requires some (very simple) changes to JS codes to make it works.

In this tutorial, we will use a simple bar chart from https://observablehq.com/@d3/bar-chart (ISC license).

### 3.1 Element
JupyterJS has an *element* property in the JS scope that is binded to the display area. Recall that *element* cannot exist in python scope because it contains classes.

In [28]:
## Original JS code 
"""
const svg = d3.create("svg")
"""

'\nconst svg = d3.create("svg")\n'

In [29]:
## This needs be changed to 
"""
const svg = d3.select(element).append("svg")
"""

'\nconst svg = d3.select(element).append("svg")\n'

### 3.2 Functions and Variables in JS Scope
To let JupyterJS maps variables and functions declared in JS scope. Those variables/functions must be explictly declared via const/var/let.

In [30]:
## Original
"""
    function BarChart(data, {
"""

'\n    function BarChart(data, {\n'

In [31]:
## Changed to 
"""
    const BarChart = function (data, {
"""

'\n    const BarChart = function (data, {\n'

Now the code becomes

In [32]:
## copy paste 
jscode = """
const BarChart = function(data, {
  x = (d, i) => i, // given d in data, returns the (ordinal) x-value
  y = d => d, // given d in data, returns the (quantitative) y-value
  title, // given d in data, returns the title text
  marginTop = 20, // the top margin, in pixels
  marginRight = 0, // the right margin, in pixels
  marginBottom = 30, // the bottom margin, in pixels
  marginLeft = 40, // the left margin, in pixels
  width = 640, // the outer width of the chart, in pixels
  height = 400, // the outer height of the chart, in pixels
  xDomain, // an array of (ordinal) x-values
  xRange = [marginLeft, width - marginRight], // [left, right]
  yType = d3.scaleLinear, // y-scale type
  yDomain, // [ymin, ymax]
  yRange = [height - marginBottom, marginTop], // [bottom, top]
  xPadding = 0.1, // amount of x-range to reserve to separate bars
  yFormat, // a format specifier string for the y-axis
  yLabel, // a label for the y-axis
  color = "currentColor" // bar fill color
} = {}) {
  // Compute values.
  const X = d3.map(data, x);
  const Y = d3.map(data, y);

  // Compute default domains, and unique the x-domain.
  if (xDomain === undefined) xDomain = X;
  if (yDomain === undefined) yDomain = [0, d3.max(Y)];
  xDomain = new d3.InternSet(xDomain);

  // Omit any data not present in the x-domain.
  const I = d3.range(X.length).filter(i => xDomain.has(X[i]));

  // Construct scales, axes, and formats.
  const xScale = d3.scaleBand(xDomain, xRange).padding(xPadding);
  const yScale = yType(yDomain, yRange);
  const xAxis = d3.axisBottom(xScale).tickSizeOuter(0);
  const yAxis = d3.axisLeft(yScale).ticks(height / 40, yFormat);

  // Compute titles.
  if (title === undefined) {
    const formatValue = yScale.tickFormat(100, yFormat);
    title = i => `${X[i]}\n${formatValue(Y[i])}`;
  } else {
    const O = d3.map(data, d => d);
    const T = title;
    title = i => T(O[i], i, data);
  }

  const svg = d3.select(element).append("svg")
      .attr("width", width)
      .attr("height", height)
      .attr("viewBox", [0, 0, width, height])
      .attr("style", "max-width: 100%; height: auto; height: intrinsic;");

  svg.append("g")
      .attr("transform", `translate(${marginLeft},0)`)
      .call(yAxis)
      .call(g => g.select(".domain").remove())
      .call(g => g.selectAll(".tick line").clone()
          .attr("x2", width - marginLeft - marginRight)
          .attr("stroke-opacity", 0.1))
      .call(g => g.append("text")
          .attr("x", -marginLeft)
          .attr("y", 10)
          .attr("fill", "currentColor")
          .attr("text-anchor", "start")
          .text(yLabel));

  const bar = svg.append("g")
      .attr("fill", color)
    .selectAll("rect")
    .data(I)
    .join("rect")
      .attr("x", i => xScale(X[i]))
      .attr("y", i => yScale(Y[i]))
      .attr("height", i => yScale(0) - yScale(Y[i]))
      .attr("width", xScale.bandwidth());

  if (title) bar.append("title")
      .text(title);

  svg.append("g")
      .attr("transform", `translate(0,${height - marginBottom})`)
      .call(xAxis);

  return svg.node();
}
"""

In [33]:
## Generate a toy data
data = [{'letter': "A", 'frequency': 0.08167}, {'letter': "B", 'frequency': 0.01492}]

In [34]:
## Now create an instance
class D3BarWidget(JupyterJS):
    def __init__(self, data, *pargs, **kwargs):
        super(D3BarWidget, self).__init__(*pargs, **kwargs)
        
        ## register data
        self.mapstate = [{'data': data}]   
        ## register the method in JS Code
        self.mapmethod = ['BarChart']  
        
        self.import_es6('import * as d3 from "https://cdn.jsdelivr.net/npm/d3@7/+esm";', 'd3')

d3bar = D3BarWidget(data = data)
d3bar.display()

In [35]:
d3bar.javascript(jscode)

Now BarChart() can be called from both js/py. However, as its parameter include functions, it is recommended to run from JS side for avoiding unnessary communications.

In [36]:
d3bar.javascript("""
BarChart(data , {
  x: d => d.letter,
  y: d => d.frequency,
  xDomain: d3.groupSort(data, ([d]) => -d.frequency, d => d.letter), // sort by descending frequency
  yFormat: "%",
  yLabel: "↑ Frequency",
  width: 200,
  height: 200,
  color: "steelblue"})
""")

Now you see the visualizations!  
![d3 bar chart](3.1.png)

### 3.3 Making Visualizations Interactive and Reactive 

Now we want to make charts interactive and reactive. Here we implement an on-hover function.   
⚠️ Reactivity is implemented using Proxy in Javascript. As such, only **objects are reactive** (not primitives such as string and numbers). A suggestion is to wrap primitives into an object such as {'value': 5}.

In [37]:
## First register on-hover event in JS
"""
   bar.on('mouseover', (e, d) => {
        hovered.value = d;
    });
"""

"\n   bar.on('mouseover', (e, d) => {\n        hovered.value = d;\n    });\n"

In [38]:
## copy paste 
jscode = """
const BarChart = function(data, {
  x = (d, i) => i, // given d in data, returns the (ordinal) x-value
  y = d => d, // given d in data, returns the (quantitative) y-value
  title, // given d in data, returns the title text
  marginTop = 20, // the top margin, in pixels
  marginRight = 0, // the right margin, in pixels
  marginBottom = 30, // the bottom margin, in pixels
  marginLeft = 40, // the left margin, in pixels
  width = 640, // the outer width of the chart, in pixels
  height = 400, // the outer height of the chart, in pixels
  xDomain, // an array of (ordinal) x-values
  xRange = [marginLeft, width - marginRight], // [left, right]
  yType = d3.scaleLinear, // y-scale type
  yDomain, // [ymin, ymax]
  yRange = [height - marginBottom, marginTop], // [bottom, top]
  xPadding = 0.1, // amount of x-range to reserve to separate bars
  yFormat, // a format specifier string for the y-axis
  yLabel, // a label for the y-axis
  color = "currentColor" // bar fill color
} = {}) {
  // Compute values.
  const X = d3.map(data, x);
  const Y = d3.map(data, y);

  // Compute default domains, and unique the x-domain.
  if (xDomain === undefined) xDomain = X;
  if (yDomain === undefined) yDomain = [0, d3.max(Y)];
  xDomain = new d3.InternSet(xDomain);

  // Omit any data not present in the x-domain.
  const I = d3.range(X.length).filter(i => xDomain.has(X[i]));

  // Construct scales, axes, and formats.
  const xScale = d3.scaleBand(xDomain, xRange).padding(xPadding);
  const yScale = yType(yDomain, yRange);
  const xAxis = d3.axisBottom(xScale).tickSizeOuter(0);
  const yAxis = d3.axisLeft(yScale).ticks(height / 40, yFormat);

  // Compute titles.
  if (title === undefined) {
    const formatValue = yScale.tickFormat(100, yFormat);
    title = i => `${X[i]}\n${formatValue(Y[i])}`;
  } else {
    const O = d3.map(data, d => d);
    const T = title;
    title = i => T(O[i], i, data);
  }
  
  const svg = d3.select(element).append("svg")
      .attr('id', 'reactived3bar')
      .attr("width", width)
      .attr("height", height)
      .attr("viewBox", [0, 0, width, height])
      .attr("style", "max-width: 100%; height: auto; height: intrinsic;");

  console.log(element, svg)

  svg.append("g")
      .attr("transform", `translate(${marginLeft},0)`)
      .call(yAxis)
      .call(g => g.select(".domain").remove())
      .call(g => g.selectAll(".tick line").clone()
          .attr("x2", width - marginLeft - marginRight)
          .attr("stroke-opacity", 0.1))
      .call(g => g.append("text")
          .attr("x", -marginLeft)
          .attr("y", 10)
          .attr("fill", "currentColor")
          .attr("text-anchor", "start")
          .text(yLabel));

  const bar = svg.append("g")
      .attr("fill", color)
    .selectAll("rect")
    .data(I)
    .join("rect")
      .attr("x", i => xScale(X[i]))
      .attr("y", i => yScale(Y[i]))
      .attr("height", i => yScale(0) - yScale(Y[i]))
      .attr("width", xScale.bandwidth());
      
   bar.on('mouseover', (e, d) => {
        hovered.value = d;
    });

  if (title) bar.append("title")
      .text(title);

  svg.append("g")
      .attr("transform", `translate(0,${height - marginBottom})`)
      .call(xAxis);

  return svg.node();
}
"""

In [39]:
data = [{'letter': "A", 'frequency': 0.08167}, {'letter': "B", 'frequency': 0.01492}]

## Now create an instance. Note that we add a new hovered variable in the state.
class ReactiveD3(JupyterJS):
    def __init__(self, data, *pargs, **kwargs):
        super(ReactiveD3, self).__init__(*pargs, **kwargs)
        
        ## register data
        self.mapstate = [{'data': data}, {'hovered': {'value': ''}}]   

        self.mapmethod = ['BarChart']  
        
        self.import_es6('import * as d3 from "https://cdn.jsdelivr.net/npm/d3@7/+esm";', 'd3')
        
d3barreactive = ReactiveD3(data = data)
d3barreactive.display()

In [40]:
d3barreactive.javascript(jscode)

In [41]:
d3barreactive.javascript("""
BarChart(data , {
  x: d => d.letter,
  y: d => d.frequency,
  xDomain: d3.groupSort(data, ([d]) => -d.frequency, d => d.letter), // sort by descending frequency
  yFormat: "%",
  yLabel: "↑ Frequency",
  width: 200,
  height: 200,
  color: "steelblue"})
""")

Now you have a reactive bar chart. Note how *hovered* changed when hovering on the bar!
![](d3barreactive.gif)